# 04 - Token Probability

This notebook demonstrates how to inspect token-level log probabilities from the OpenAI API, showing the model's confidence for each generated token and its top alternatives.

**Prerequisites:** Set the `OPENAI_API_KEY` environment variable before running.

In [ ]:
import math
from openai import AzureOpenAI, OpenAI
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Structured Output with Logprobs

Request a structured joke response with `logprobs=True` and `top_logprobs=5` to see token-level probabilities.

In [ ]:
class Joke(BaseModel):
    joke: str = Field(description="The joke.")
    explanation: str = Field(description="The explanation of the joke.")


completion = client.beta.chat.completions.parse(
    model="gpt-5.1",
    messages=[
        {"role": "system", "content": "You are an joke writing assistant"},
        {"role": "user", "content": "Tell me a joke about pirates"},
    ],
    response_format=Joke,
    logprobs=True,
    top_logprobs=5,
)

choice = completion.choices[0]

joke = choice.message.parsed.joke
print(joke)

## Inspect Token Probabilities

For each token in the response, show the probability of the chosen token and its top-5 alternatives.

In [ ]:
for token_info in choice.logprobs.content:
    token = token_info.token
    logp = token_info.logprob
    p = math.exp(logp)

    print(f"token={repr(token):10} p={p:.6f}")
    for alt in token_info.top_logprobs:
        alt_p = math.exp(alt.logprob)
        print(f"  alt={repr(alt.token):10} p={alt_p:.6f}")